In [1]:

import nbimporter

# from Database_connection.database_connection import connect_db
import requests
import pandas as pd
import numpy as np
from sqlalchemy import MetaData, create_engine
from datetime import datetime

In [2]:
# Temporal db Connection (Fix Route)
def connect_db(database = 'coins'):
    user='root'
    password='240699'
    host='localhost'
    database = database
    

    # Connect to the database
    engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")
    try:
        connection = engine.connect()
        return connection
    except Exception as e:
        raise e

## From each Protocol: Get its Historic TVL

In [3]:
response = requests.get('https://api.llama.fi/protocols')
data = response.json()
df = pd.DataFrame.from_dict(data, orient = 'Columns')
names = df.slug

In [8]:
names

0       binance-cex
1              lido
2          bitfinex
3               okx
4         robinhood
           ...     
3574        tokenfi
3575    spirit-farm
3576         aitech
3577           hopr
3578           boom
Name: slug, Length: 3579, dtype: object

## This is not historic. It's just actual tvl on each diferent chain that the protocol has.

### Same as db table protocols_tvl.by_chain

In [4]:
failed = 0
count = 0
protocols_failed = []
try:
    with connect_db('protocols_historic_tvl_by_chain') as connection:
        for coin in names:
            try:
                response = requests.get(f'https://api.llama.fi/protocol/{coin}')
                response.raise_for_status()  # Raise an HTTPError for bad responses
                data = response.json()

                chains = data['currentChainTvls']

                # Create DataFrame without index
                assetDf = pd.DataFrame.from_dict(chains, orient='index', columns=['tvl']).reset_index()

                # Add a column for 'coin'
                assetDf['coin'] = coin

                # Define the table name
                table_name = f'{coin}'

                # Write DataFrame to SQL table without including the index
                assetDf.to_sql(table_name, con=connection, if_exists='replace', index=False)
                count+=1
                print(f'{coin} inserted { len(names) - count } to go.')

            except requests.exceptions.RequestException as req_exception:
                print(f'Error fetching data for {coin}: {req_exception}')
                failed+=1
                protocols_failed.append(coin)

            except Exception as ex:
                print(f'Error processing {coin}: {ex}')
                protocols_failed.append(coin)
                failed+=1  

except Exception as e:
    print(f'Error connecting to the database: {e}')
    failed+=1
print (failed)

binance-cex inserted 3738 to go.
lido inserted 3737 to go.
bitfinex inserted 3736 to go.
okx inserted 3735 to go.
robinhood inserted 3734 to go.
polygon-bridge-&-staking inserted 3733 to go.
wbtc inserted 3732 to go.
justlend inserted 3731 to go.
arbitrum-bridge inserted 3730 to go.
bybit inserted 3729 to go.
makerdao inserted 3728 to go.
justcryptos inserted 3727 to go.
crypto-com inserted 3726 to go.
kraken inserted 3725 to go.
aave-v3 inserted 3724 to go.
huobi inserted 3723 to go.
rocket-pool inserted 3722 to go.
deribit inserted 3721 to go.
summer.fi inserted 3720 to go.
bitstamp inserted 3719 to go.
maker-rwa inserted 3718 to go.
kucoin inserted 3717 to go.
uniswap-v3 inserted 3716 to go.
aave-v2 inserted 3715 to go.
stusdt inserted 3714 to go.
optimism-bridge inserted 3713 to go.
instadapp inserted 3712 to go.
gate-io inserted 3711 to go.
convex-finance inserted 3710 to go.
binance-staked-eth inserted 3709 to go.
bitget inserted 3708 to go.
spark inserted 3707 to go.
curve-dex i

In [12]:
assetDf

,index,tvl,coin
0,Tron,5.885887e+09,justlend
1,Tron-borrowed,1.002692e+08,justlend
2,borrowed,1.002692e+08,justlend


## Get HIstoric TVL on each chain by Protocol

In [15]:
failed = 0
count = 0
timestamp = datetime.now()

try:
    with connect_db('protocols_historic_tvl_by_chain') as connection:
        for coin in names:
            try:
                response = requests.get(f'https://api.llama.fi/protocol/{coin}')
                response.raise_for_status()  # Raise an HTTPError for bad responses
                data = response.json()
                tvl = data['chainTvls']

                assetChains = []
                dataList = []
                for key, value in tvl.items():
                    assetChains.append(key)
                    for innerKey, innerValue in value.items():
                        if innerValue is None:
                            continue
                        for differentDay in innerValue:
                            date = differentDay['date']
                            if 'totalLiquidityUSD' in differentDay:
                                total_liquidity = differentDay['totalLiquidityUSD']
                                chain = key
                                dataList.append({'date': date, 'tvl': total_liquidity, 'chain': chain, 'protocol': coin})

                df = pd.DataFrame(dataList)
                df['date'] = pd.to_datetime(df['date'], unit='s')
                df['updated_at'] = timestamp # not exactly the same time for each but its faster, less calculations.
                table_name = coin
                df.to_sql(table_name, con=connection, if_exists='replace')
                count += 1
                print(f'Inserted {coin}! - {len(names) - count} to go')

            except requests.exceptions.RequestException as req_exception:
                print(f'Error fetching data for {coin}: {req_exception}')
                failed += 1

            except Exception as ex:
                print(f'Error processing {coin}: {ex}')
                failed += 1

except Exception as e:
    print(f'Error connecting to the database: {e}')
    failed += 1

print(f'Number of failures: {failed}')

                
    

Inserted binance-cex! - 3578 to go
Inserted lido! - 3577 to go
Inserted bitfinex! - 3576 to go
Inserted okx! - 3575 to go
Inserted robinhood! - 3574 to go
Inserted polygon-bridge-&-staking! - 3573 to go
Inserted wbtc! - 3572 to go
Inserted justlend! - 3571 to go
Inserted makerdao! - 3570 to go
Inserted arbitrum-bridge! - 3569 to go
Inserted bybit! - 3568 to go
Inserted justcryptos! - 3567 to go
Inserted crypto-com! - 3566 to go
Inserted kraken! - 3565 to go
Inserted huobi! - 3564 to go
Inserted aave-v3! - 3563 to go
Inserted maker-rwa! - 3562 to go
Inserted summer.fi! - 3561 to go
Inserted deribit! - 3560 to go
Inserted aave-v2! - 3559 to go
Inserted stusdt! - 3558 to go
Inserted rocket-pool! - 3557 to go
Inserted uniswap-v3! - 3556 to go
Inserted kucoin! - 3555 to go
Inserted gate-io! - 3554 to go
Inserted bitstamp! - 3553 to go
Inserted instadapp! - 3552 to go
Inserted convex-finance! - 3551 to go
Inserted juststables! - 3550 to go
Inserted curve-dex! - 3549 to go
Inserted bitget! - 

In [ ]:
# 54 / 3578 not inserted Time: 84m 32s

## Same code optimized by chatGpt


In [ ]:
failed = 0
count = 0
timestamp = datetime.now()

try:
    with connect_db('protocols_historic_tvl_by_chain') as connection:
        for coin in names:
            try:
                response = requests.get(f'https://api.llama.fi/protocol/{coin}')
                response.raise_for_status()  # Raise an HTTPError for bad responses
                data = response.json()
                tvl = data['chainTvls']

                # Extract data using list comprehension
                coin_data = [
                    {'date': differentDay['date'],
                     'tvl': differentDay.get('totalLiquidityUSD', None),
                     'chain': key,
                     'protocol': coin}
                    for key, value in tvl.items() if value is not None
                    for innerKey, innerValue in value.items() if innerValue is not None
                    for differentDay in innerValue
                ]

                # Create a DataFrame for the current coin
                df = pd.DataFrame(coin_data)
                df['date'] = pd.to_datetime(df['date'], unit='s')
                df['updated_at'] = timestamp

                # Write the DataFrame to a table named after the coin
                df.to_sql(coin, con=connection, if_exists='replace', index=False)

                count += 1
                print(f'Inserted {coin}! - {len(names) - count} to go')

            except requests.exceptions.RequestException as req_exception:
                print(f'Error fetching data for {coin}: {req_exception}')
                failed += 1

            except Exception as ex:
                print(f'Error processing {coin}: {ex}')
                failed += 1

except requests.exceptions.RequestException as e:
    print(f'Error connecting to the database: {e}')
    failed += 1

print(f'Number of failures: {failed}')


In [ ]:
failed = 0
count = 0
timestamp = datetime.now()

try:
    with connect_db('protocols_historic_tvl_by_chain') as connection:
        all_data = []  # Collect data for all coins

        for coin in names:
            try:
                response = requests.get(f'https://api.llama.fi/protocol/{coin}')
                response.raise_for_status()  # Raise an HTTPError for bad responses
                data = response.json()
                tvl = data['chainTvls']

                # Extract data using list comprehension
                coin_data = [
                    {'date': differentDay['date'],
                     'tvl': differentDay.get('totalLiquidityUSD', None),
                     'chain': key,
                     'protocol': coin}
                    for key, value in tvl.items() if value is not None
                    for innerKey, innerValue in value.items() if innerValue is not None
                    for differentDay in innerValue
                ]

                all_data.extend(coin_data)

                count += 1
                print(f'Fetched data for {coin}! - {len(names) - count} to go')

            except requests.exceptions.RequestException as req_exception:
                print(f'Error fetching data for {coin}: {req_exception}')
                failed += 1

            except Exception as ex:
                print(f'Error processing {coin}: {ex}')
                failed += 1

        # Combine all data into a single DataFrame
        df = pd.DataFrame(all_data)
        df['date'] = pd.to_datetime(df['date'], unit='s')
        df['updated_at'] = timestamp

        # Write the combined DataFrame to SQL
        df.to_sql('protocols_data', con=connection, if_exists='replace', index=False)

except requests.exceptions.RequestException as e:
    print(f'Error connecting to the database: {e}')
    failed += 1

print(f'Number of failures: {failed}')
